In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, Model
from scipy.io import loadmat

class PINN(Model):
    def __init__(self, layers_dims):
        super(PINN, self).__init__()
        self.network = tf.keras.Sequential()
        for dim in layers_dims:
            self.network.add(layers.Dense(dim, activation="tanh"))
        self.network.add(layers.Dense(1))  

    def call(self, inputs):
        return self.network(inputs)

In [2]:
def physics_informed_loss(
    model, x, t, initial_condition, boundary_condition, epsilon
):
    with tf.GradientTape(persistent=True) as tape:
        tape.watch([x, t])
        u = model(tf.concat([x, t], axis=1))

        u_t = tape.gradient(u, t)
        u_x = tape.gradient(u, x)
        u_xx = tape.gradient(u_x, x)

    residual = u_t + u * u_x - epsilon * u_xx

    # Initial condition loss
    ic_loss = model(initial_condition[:, :2])
    initial_loss = tf.reduce_mean(tf.square(ic_loss - initial_condition[:, 2]))

    # Periodic boundary condition loss
    bc_loss = model(boundary_condition[:, :2])
    bc_loss = tf.reduce_mean(tf.square(bc_loss - boundary_condition[:, 2]))

    # Residual loss
    residual_loss = tf.reduce_mean(tf.square(residual))

    return initial_loss + bc_loss + residual_loss

In [3]:
def train_model(
    model,
    x,
    t,
    initial_condition,
    boundary_condition,
    du,
    epsilon,
    epochs,
    learning_rate,
):
    x_grid, t_grid = tf.meshgrid(x[:, 0], t[:, 0])  
    x_flat = tf.reshape(x_grid, [-1, 1])  
    t_flat = tf.reshape(t_grid, [-1, 1]) 

    optimizer = tf.keras.optimizers.Adam(learning_rate)

    for epoch in range(epochs):
        with tf.GradientTape() as tape:
            loss = physics_informed_loss(
                model,
                x_flat,
                t_flat,
                initial_condition,
                boundary_condition,
                du,
                epsilon,
            )
        grads = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(grads, model.trainable_variables))

        if epoch % 100 == 0:
            print(f"Epoch {epoch}, Loss: {loss.numpy():.6f}")

In [4]:
def load_data(file_name, type):
    with h5py.File(file_name, "r") as f:
        initial_condition = f[f"0/initial_condition_{type}"][:]
        boundary_condition = f[f"0/boundary_condition_{type}"][:]
        clean_data = f["0/clean"][:]
        du = f["0/du"][()]
        epsilon = f["0/epsilon"][()]
        u0 = f["0/u0"][()]
        x = f["coords/x-coordinates"][:]
        t = f["coords/t-coordinates"][:-1]
    return (
        initial_condition,
        boundary_condition,
        clean_data,
        du,
        epsilon,
        u0,
        x,
        t,
    )

In [5]:
# Load the data
file_name = (
    "/kaggle/input/burgers-noisy/simulation_data.h5"  # Replace with your data file
)
initial_condition, boundary_condition, clean_data, du, epsilon, u0, x, t = load_data(
    file_name, "noisy"
)

x = tf.convert_to_tensor(x, dtype=tf.float32)[:, None]  # x-coordinates as column vector
t = tf.convert_to_tensor(t, dtype=tf.float32)[:, None]  # t-coordinates as column vector
initial_condition = tf.convert_to_tensor(initial_condition, dtype=tf.float32)
boundary_condition = tf.convert_to_tensor(boundary_condition, dtype=tf.float32)

layers_dims = [50, 50, 50]
epochs = 10000
learning_rate = 1e-3

model = PINN(layers_dims)

## Training Noisy

In [6]:
train_model(
    model,
    x,
    t,
    initial_condition,
    boundary_condition,
    du,
    epsilon,
    epochs,
    learning_rate,
)

Epoch 0, Loss: 1.494818
Epoch 100, Loss: 0.554695
Epoch 200, Loss: 0.329819
Epoch 300, Loss: 0.261835
Epoch 400, Loss: 0.251033
Epoch 500, Loss: 0.243114
Epoch 600, Loss: 0.235674
Epoch 700, Loss: 0.225637
Epoch 800, Loss: 0.210556
Epoch 900, Loss: 0.192006
Epoch 1000, Loss: 0.179609
Epoch 1100, Loss: 0.172866
Epoch 1200, Loss: 0.168310
Epoch 1300, Loss: 0.166577
Epoch 1400, Loss: 0.165576
Epoch 1500, Loss: 0.164757
Epoch 1600, Loss: 0.165785
Epoch 1700, Loss: 0.163406
Epoch 1800, Loss: 0.162896
Epoch 1900, Loss: 0.162403
Epoch 2000, Loss: 0.161876
Epoch 2100, Loss: 0.161491
Epoch 2200, Loss: 0.161090
Epoch 2300, Loss: 0.160681
Epoch 2400, Loss: 0.160311
Epoch 2500, Loss: 0.159940
Epoch 2600, Loss: 0.159551
Epoch 2700, Loss: 0.159180
Epoch 2800, Loss: 0.158714
Epoch 2900, Loss: 0.158261
Epoch 3000, Loss: 0.157784
Epoch 3100, Loss: 0.157315
Epoch 3200, Loss: 0.156725
Epoch 3300, Loss: 0.156198
Epoch 3400, Loss: 0.155630
Epoch 3500, Loss: 0.155062
Epoch 3600, Loss: 0.154460
Epoch 3700, L

In [10]:
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import animation
from tqdm import tqdm


def visualize_burgers(xcrd, data, path):
    """
    This function animates the Burgers equation

    Args:
    path : path to the desired file
    param: PDE parameter of the data shard to be visualized
    """
    fig, ax = plt.subplots()
    ims = []

    for i in tqdm(range(data.shape[0])):
        if i == 0:
            im = ax.plot(xcrd, data[i].squeeze(), animated=True, color="blue")
        else:
            im = ax.plot(
                xcrd, data[i].squeeze(), animated=True, color="blue"
            ) 
        ims.append([im[0]])

    ani = animation.ArtistAnimation(fig, ims, interval=50, blit=True, repeat_delay=1000)

    writer = animation.PillowWriter(fps=15, bitrate=1800)
    ani.save(path, writer=writer)
    plt.close(fig)
    
visualize_burgers(x, predicted_solution, "noisy_trained_predicted_solution.gif")

100%|██████████| 201/201 [00:00<00:00, 353.87it/s]
